### **Problem 4 :** Quantum Teleportation
The goal of quantum teleportation is to transfer the state of one qubit to another, using entanglement and classical communication.

Concept:
1. Entanglement: Create an entangled pair of qubits, A and B.
2. State Preparation: Prepare a third qubit, Q, in an arbitrary quantum state (the state you want to "teleport").
3. Measurement and Communication:
    - Measure Q and A in a specific basis (using CNOT and Hadamard gates).
    - Send the results of the measurement (classical bits) to the owner of B.

4. Reconstruction:
    - Based on the received classical bits, apply appropriate gates to 
B to recreate Q's state.


Circuit Overview :
We’ll create a 3-qubit circuit:
- Q: The qubit to be teleported.
- A: Entangled with 
- B: Receives Q's state after teleportation.

In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.compiler import transpile

In [2]:
# Create quantum and classical registers
q = QuantumRegister(3, 'q')  # 3 qubits
c = ClassicalRegister(2, 'c')  # 2 classical bits

In [3]:
# Create quantum circuit with registers
qc = QuantumCircuit(q, c)

# Prepare the state to teleport (arbitrary state on Q)
qc.h(q[0])  # Create a superposition state on Q
qc.p(3.14 / 4, q[0])  # Add a phase for complexity

In [4]:
# Create entanglement between qubits A and B
qc.h(q[1])  # Hadamard gate on A
qc.cx(q[1], q[2])  # CNOT between A (control) and B (target)

# Entangle Q with A and measure
qc.cx(q[0], q[1])  # CNOT between Q (control) and A (target)
qc.h(q[0])  # Hadamard gate on Q


In [5]:
# Measure Q and A
qc.measure(q[0], c[0])  # Measure Q (stored in classical bit 0)
qc.measure(q[1], c[1])  # Measure A (stored in classical bit 1)

In [6]:
# Quantum teleportation correction based on measurement
# Modern approach for conditional operations
qc.x(q[2]).c_if(c, 1)  # Apply X gate if first classical bit is 1
qc.z(q[2]).c_if(c, 2)  # Apply Z gate if second classical bit is 1


/tmp/ipykernel_11997/530418690.py:3: DeprecationWarning: The method ``qiskit.circuit.instructionset.InstructionSet.c_if()`` is deprecated as of qiskit 1.3.0. It will be removed in 2.0.0.
  qc.x(q[2]).c_if(c, 1)  # Apply X gate if first classical bit is 1
/tmp/ipykernel_11997/530418690.py:4: DeprecationWarning: The method ``qiskit.circuit.instructionset.InstructionSet.c_if()`` is deprecated as of qiskit 1.3.0. It will be removed in 2.0.0.
  qc.z(q[2]).c_if(c, 2)  # Apply Z gate if second classical bit is 1


In [7]:
# Simulator
simulator = Aer.get_backend('qasm_simulator')


In [8]:
# Transpile and run the circuit
transpiled_circuit = transpile(qc, simulator)
job = simulator.run(transpiled_circuit, shots=1000)
result = job.result()


In [9]:
# Get results
counts = result.get_counts(qc)
print("Measurement Results:", counts)

Measurement Results: {'11': 243, '00': 238, '10': 259, '01': 260}


In [10]:
# Circuit visualization
qc.draw()

┌───┐┌──────────┐     ┌───┐┌─┐              
q_0: ┤ H ├┤ P(0.785) ├──■──┤ H ├┤M├──────────────
     ├───┤└──────────┘┌─┴─┐└┬─┬┘└╥┘              
q_1: ┤ H ├─────■──────┤ X ├─┤M├──╫───────────────
     └───┘   ┌─┴─┐    └───┘ └╥┘  ║  ┌───┐  ┌───┐ 
q_2: ────────┤ X ├───────────╫───╫──┤ X ├──┤ Z ├─
             └───┘           ║   ║  └─╥─┘  └─╥─┘ 
                             ║   ║ ┌──╨──┐┌──╨──┐
c: 2/════════════════════════╩═══╩═╡ 0x1 ╞╡ 0x2 ╞
                             1   0 └─────┘└─────┘